In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import random

import sys
sys.path.append("../")
sys.path.append("../../src")
sys.path.append("../../")

from src.train import Trainer
from src.nn.net import get_new_net, load_net
from src.data.plot_light_curve import plot_curves
from src.config import Config, DataConfig, FilterConfig, PACKAGE_PATH, BasicCNNConfig, NetConfig, FCConfig

In [2]:
import os
FOLDER_NAME = "Fourier_FC_9_2_2023_3000"
for f in ["models", "datasets","configurations"]:
    os.makedirs(f"{PACKAGE_PATH}/output/{f}/{FOLDER_NAME}", exist_ok=True)

In [3]:
net_config = NetConfig(
        name=f"{FOLDER_NAME}_v3.1",
        net_class="FCNet",
        input_size=17,
        n_classes=5,
        device="cuda:0",
        save_path=f"{PACKAGE_PATH}/output/models/{FOLDER_NAME}/",
        net_args=FCConfig(
            layers=[256]
        ).__dict__
)

data_config = DataConfig(
        path=f"{PACKAGE_PATH}/resources/mmt_13_3_2023/",
        labels=["cz_3", "falcon_9", "atlas",  "h2a", "globalstar"],
        convert_to_mag=False,
        number_of_training_examples_per_class = 3000,
        validation_split = 0.2,
        dataset_class="FourierDataset",
        dataset_arguments={},
        filter=FilterConfig(
            n_bins= 30,
            n_gaps= 10,
            gap_size= 5, 
            rms_ratio= 0.,
            non_zero_ratio= 0.8
        )
)

cfg = Config(net_config=net_config, data_config=data_config)


SAMPLER = True

LOAD = False
SEED = None
DATA_SEED = f"{cfg.data_config.filter.n_bins}_{cfg.data_config.filter.n_gaps}_{cfg.data_config.filter.gap_size}_{int(cfg.data_config.filter.non_zero_ratio * 10)}"
CHECKPOINT = "latest"

In [4]:
trainer = Trainer(None)
net = None

dataset_path = f"{PACKAGE_PATH}/output/datasets/{FOLDER_NAME}"

if os.path.exists(f"{dataset_path}/{DATA_SEED}"):
    trainer.load_data_from_file(f"{dataset_path}/{DATA_SEED}")
else:
    trainer.load_data(cfg.data_config)
    os.makedirs(f"{dataset_path}/{DATA_SEED}", exist_ok=True)
    trainer.save_data(f"{dataset_path}/{DATA_SEED}")



if LOAD:
    net = load_net(cfg, seed=SEED, checkpoint=CHECKPOINT)
    trainer.net = net
    trainer.load_data_from_file(f"{dataset_path}/{DATA_SEED}")
else:
    net = get_new_net(cfg)
    with open(f"{PACKAGE_PATH}/output/configurations/{FOLDER_NAME}/{net.name}.json", "w") as f:
        print(cfg.to_json(), file=f)
    SEED = cfg.seed
    trainer.net = net
    
if SAMPLER:
    trainer.add_sampler()

SEED: 989980


In [5]:
SEED

989980

In [6]:
str(net)

'FCNet(\n  (layers): Sequential(\n    (0): Linear(in_features=17, out_features=256, bias=True)\n    (1): ReLU()\n    (2): Linear(in_features=256, out_features=5, bias=True)\n  )\n  (logsoftmax): LogSoftmax(dim=1)\n)'

In [7]:
trainer.train(10, 64,tensorboard_on=True, save_interval=1001, print_on=False)

Training: 100%|██████████| 10/10 [00:05<00:00,  1.74it/s]


In [8]:
trainer.net.epoch_trained

10

In [9]:
trainer.evaluate(cfg.data_config.labels, save_path='test.csv')

Train:
	Loss: 0.01713487061216233
	Acc: 52.89520127741827
Validation:
	Loss: 0.023009728323240514
	Acc: 20.705414803775458
-----------------------------------------

        Label  cz_3  falcon_9  atlas  h2a  globalstar
0        cz_3    19      4804   1679  628          39
1    falcon_9     0       376     35   29           1
2       atlas     0       251    193   68          40
3         h2a     0        46     13  421           3
4  globalstar     0        29    111  205        1075

-----------------------------------------

                 cz_3   falcon_9      atlas        h2a  globalstar
Precision    0.265030  85.260771  34.963768  87.163561   75.704225
Recall     100.000000   6.828914   9.502708  31.162102   92.832470
F1 score     0.528659  12.645031  14.943864  45.910578   83.397983

-----------------------------------------

